In [0]:
%run ./0.Config

In [0]:
def load_data():
    vendor_df = spark.read.format("csv").option("header", "true").load(VENDOR_DATA_PATH)
    return vendor_df.toPandas()

def setup_llm():
    return ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, model_kwargs={"temperature": 0.2})

def save_zone_csv(zone):
    path = f"/dbfs/FileStore/zone_vendor_files/{zone}.csv"
    return path   

def summarize(path):
    df = pd.read_csv(path)
    routes = sorted(df["Route_ID"].unique())
    truck_types = sorted(df["Truck_Type"].unique())
    total_count = int(df["Required_Trucks"].sum())
    return routes, truck_types, total_count

def generate_email(llm, contact_name, zone, routes, truck_types, total_count, vendor_name):
    template = PromptTemplate(
        input_variables=[
            "contact_name", "zone", "routes", "truck_types", "total_count",
            "vendor_name", "application_link", "sender_name", "sender_contact"
        ],
        template=(
            "You are a logistics procurement officer. Draft a formal RFQ email to {contact_name} from {vendor_name} "
            "regarding truck procurement in the {zone} region.\n\n"
            "Include:\n"
            "- Routes: {routes}\n"
            "- Truck Types: {truck_types}\n"
            "- Total truck requirement: {total_count}\n"
            "- Mention that a CSV file with detailed requirements is attached.\n"
            "- Mention vendors should fill out the application form at: {application_link}\n"
            "- Deadline to apply is within 7 days of receiving the email.\n"
            "- Sign off with {sender_name} and {sender_contact}.\n\n"
            "**Do not include introductory lines like 'Here is the email'. Just output the message as specified.**\n"
            "Output format:\n"
            "Subject: <subject line>\n\n"
            "Dear {contact_name},\n\n<Body Text>\nBest regards,\nCoca Cola Pvt Ltd\nprocurement@cocacola.com"
        )
    )
    chain = template | llm
    inputs = {
        "contact_name": contact_name,
        "zone": zone,
        "routes": ", ".join(routes),
        "truck_types": ", ".join(truck_types),
        "total_count": total_count,
        "vendor_name": vendor_name,
        "application_link": APPLICATION_LINK,
        "sender_name": SENDER_NAME,
        "sender_contact": SENDER_CONTACT
    }
    output = chain.invoke(inputs)
    if hasattr(output, "content"):  
        output = output.content
        
    subject = re.search(r"Subject:\s*(.*)", output)
    subject_line = subject.group(1).strip() if subject else "Request for Quotation"
    body = re.sub(r"Subject:.*\n?", "", output).strip()
    return subject_line, body

def send_email(recipient, subject, body, attachment1):
    try:
        yag = yagmail.SMTP(user=EMAIL_USER, password=EMAIL_PASSWORD)
        yag.send(to=recipient, subject=subject, contents=body, attachments=[attachment1])
        return f"Email sent to {recipient}"
    except Exception as e:
        return f"Failed to send email to {recipient}: {str(e)}"

def rfq_email_tool_function(_: str) -> str:
    logs = []
    try:
        vendor_df = load_data()
        llm = setup_llm()

        for _, vendor in vendor_df.iterrows():
            try:
                zone = vendor["Region"]
                contact_name = vendor["Vendor Name"]
                email = vendor["Vendor Email"]
                vendor_name = vendor["Vendor Name"]

                zone_csv = save_zone_csv(zone)
                routes, truck_types, total = summarize(zone_csv)
                subject, body = generate_email(
                    llm, contact_name, zone, routes, truck_types, total, vendor_name
                )
                result = send_email(email, subject, body, zone_csv)
                logs.append(result)

            except Exception as vendor_err:
                logs.append(f"Error processing vendor {vendor.get('Vendor ID', 'N/A')}: {str(vendor_err)}")

    except Exception as e:
        logs.append(f"Pipeline failed: {str(e)}")

    return "\n".join(logs)

# Register as a LangChain Tool
rfq_email_tool = Tool(
    name="rfq_email_tool",
    func=rfq_email_tool_function,
    description="Sends RFQ emails to vendors using forecast and vendor CSV data, attaches zone-wise truck requirement CSVs, and uses LLM to generate email body."
)

In [0]:
tools = [rfq_email_tool]

agent = initialize_agent(
    tools=tools,
    llm=ChatDatabricks(endpoint="databricks-llama-4-maverick"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Sends RFQ emails to vendors.")